In [ ]:
%cd ..

In [ ]:
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("ggplot")

In [ ]:
feedback_dir = Path("data", "raw", "feedback")
feedback_paths = {
    re.search("(?<=Feedback on the ).*?(?= fluency)", path.name).group(): path
    for path in feedback_dir.glob("Feedback*.xlsx")
}
print(f"Loaded {len(feedback_paths):,} feedback paths.")
feedback_paths

In [ ]:
results = {
    language: pd.read_excel(path).iloc[:, 6:].values
    for language, path in feedback_paths.items()
}
results = dict(sorted(results.items(), key=lambda x: x[1].mean()))

In [ ]:
y_pos = np.arange(len(results))
means = [matrix.mean() for matrix in results.values()]
ci_errors = [
    1.96 * matrix.std(ddof=1) / np.sqrt(matrix.size) for matrix in results.values()
]

plt.figure(figsize=(14, 7))
plt.errorbar(
    x=means,
    y=y_pos,
    xerr=ci_errors,
    fmt="o",  # 'o' for a marker at the mean
    color="black",
    capsize=5,  # Adds caps to the error bars
    linestyle="None",  # No line connecting the means
    elinewidth=2,  # Thickness of the error bar lines
    markeredgecolor="black",
    markerfacecolor="skyblue",
    markersize=8,
    label="Mean value with 95% confidence interval",
)
plt.legend(fontsize=11)
plt.xticks(np.arange(1, 3.01, 0.1))
plt.yticks(
    ticks=y_pos,
    labels=[f"{language} (n={matrix.size})" for language, matrix in results.items()],
    fontsize=10,
)
plt.xlabel("Rating (1-3)")
plt.title("Fluency Rating of LLM-generated Questions per Language", fontsize=15)
plt.savefig(
    "data/processed/fluency-rating-plot.png",
    dpi=300,
    bbox_inches="tight",
    pad_inches=0.1,
)